In [1]:
 # connect to ssh
import pandas as pd
import pymysql
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder

In [2]:
# SSH connection vars
ssh_host = 'IP_ADDRESS'
ssh_username = 'USER'
ssh_password = 'PASS'

In [3]:
def open_ssh_tunnel(verbose=False):
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global tunnel
    tunnel = sshtunnel.SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_password = ssh_password,
        remote_bind_address = ('127.0.0.1', 3306)
    )
    
    tunnel.start()


In [4]:
def mysql_connect():
    global connection
    
    connection = pymysql.connect(
        host ='127.0.0.1',
        user = db_username,
        passwd = db_password,
        db = db_name,
        port = tunnel.local_bind_port
    )

In [5]:
def run_query(sql):
    return pd.read_sql_query(sql, connection)

In [6]:
def mysql_disconnect():
    connection.close()

In [7]:
def close_ssh_tunnel():
    tunnel.close

In [9]:
open_ssh_tunnel()
mysql_connect()
tables = run_query("SHOW TABLES;")
tables_dfs = []

for name in tables.values:
    table = name[0]
    query = f"SELECT * FROM {table};"
    tables_dfs.append({
        'Name': name,
        'Table': run_query(query)
    })

mysql_disconnect()
close_ssh_tunnel()